In [ ]:
import os

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings

warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("csv/train.csv")

In [ ]:
print(df.head())
print(df.info())

In [ ]:
df = df.drop(['store', 'item'], axis=1)

In [ ]:
print(df.info())

CONVERTER DATA DE OBJETO PRA DATETIME DATATYPE ANIMAL

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.info()

converter dias para mês(bekbrace)

In [ ]:
df['date'] = df['date'].dt.to_period('M')
Vendas_mensais = df.groupby('date').sum().reset_index()

convertendo pra linha do tempo

In [ ]:
df['date'] = Vendas_mensais['date'].dt.to_timestamp()

In [ ]:
Vendas_mensais.head(10)

Visualização

In [ ]:
plt.Figure(figsize=(12,6))
plt.plot(Vendas_mensais['date'].dt.to_timestamp(), Vendas_mensais['sales'])
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('Vendas mensais pelo tempo')
plt.show()

chamando diferença pra dados estácionarios

In [ ]:
Vendas_mensais['vendas_diff'] = Vendas_mensais['sales'].diff()
Vendas_mensais = Vendas_mensais.dropna()
Vendas_mensais.head(10)

^^^ fazendo a diferença do mes anterior
617382 - 459417 = 157965

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(Vendas_mensais['date'].dt.to_timestamp(), Vendas_mensais['sales'])
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('diferença de vendas pelo tempo')
plt.show()

preparando data pra treinamento

In [ ]:
supervised_data = Vendas_mensais.drop(['date', 'sales'], axis=1)

In [ ]:
for i in range(1, 13):
    col_name = 'mes' + str(i)
    supervised_data[col_name] = supervised_data['vendas_diff'].shift(i)
supervised_data = supervised_data.dropna().reset_index(drop=True)
supervised_data.head(10)

dados de treinamento e validação

In [ ]:
Treino_data = supervised_data[:-12]
Val_data = supervised_data[-12:]
print("Treino shape: ", Treino_data.shape)
print("Validação shape: ", Val_data.shape)

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(Treino_data)
Treino_data = scaler.transform(Treino_data)
Val_data = scaler.transform(Val_data)

In [ ]:
X_Treino, Y_Treino = Treino_data[:,1:], Treino_data[:,0:]
X_Val, Y_Val = Val_data[:,1:], Val_data[:,0:1]
Y_Treino = Y_Treino.ravel()
Y_Val = Y_Val.ravel()
print("X_Treino shape: ", X_Treino.shape)
print("Y_Treino shape: ", Y_Treino.shape)
print("X_Val shape: ", X_Val.shape)
print("Y_Val shape: ", Y_Val.shape)

dataframe de predição pra dar merge nas predições de vendas dos algoritmos anterioires

In [ ]:
dates_for_pred = Vendas_mensais['date'][-12:].reset_index(drop=True)
df_pred = pd.DataFrame(dates_for_pred)

valor pra comparação futura

In [ ]:
print(df.columns)

In [ ]:
Vendas_reais = Vendas_mensais['sales'][-13:].to_list()
print(Vendas_reais)